In [1]:
import sys
sys.path.append('../')

In [2]:
from tabnet.utils.logger import init_logger
from tabnet.estimator import TabNetRegressor
from sklearn.datasets import load_boston

In [3]:
logger_dir = 'logs'
logger_name = 'TestRegression'
level = 'INFO'

logger = init_logger(logger_dir=logger_dir, logger_name=logger_name, level=level)

In [4]:
X, y = load_boston(return_X_y=True)

print(X.shape)
print(y.shape)

(506, 13)
(506,)


In [5]:
tabnet = TabNetRegressor(
    input_dims=13, output_dims=[1], logger=logger, is_cuda=False,
    reprs_dims=4, atten_dims=4, num_steps=3, num_indep=1, num_shared=1
)

In [6]:
tabnet.build(path=None)

[2021-02-19 08:49:56,149][WARNING][TabNet] Failed to load model from None


TabNetRegressor(atten_dims=4, batch_size=1024, cate_dims=None,
                cate_embed_dims=1, cate_indices=None, criterions=['mse'],
                gamma=1.3, input_dims=13, is_cuda=False, is_shuffle=True,
                logger=<RootLogger root (INFO)>, mask_type='sparsemax',
                momentum=0.03, num_indep=1, num_shared=1, num_steps=3,
                num_workers=4, output_dims=[1], pin_memory=True, reprs_dims=4,
                task_weights=1, virtual_batch_size=128)

In [ ]:
from torch.optim import Adam
from torch.optim import lr_scheduler


training_params = {
    'batch_size': 256,
    'max_epochs': 200,
    'metrics': ['mse'],
    'optimizer': Adam,
    'optimizer_params': {'lr': 0.1},
    'schedulers': [lr_scheduler.ExponentialLR],
    'scheduler_params': {'gamma': 0.99}
}


In [ ]:
tabnet.fit(X, y.reshape(-1, 1), **training_params)

In [ ]:
importance, masks = tabnet.explain(X)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, axs = plt.subplots(4, 1, figsize=(5,15))

for i in range(3):
    axs[i].imshow(masks[i].cpu().numpy()[:20])
#     axs[i].set_xlabel('features')
    axs[i].set_ylabel('samples')
    axs[i].set_title(f"mask {i}")
    axs[i].set_yticks(range(20))
#     axs[i].set_xticks(range(30))
#     axs[i].set_xticklabels(feature_names, rotation=90)

axs[3].imshow(importance.cpu().numpy()[:20, :])
axs[3].set_xlabel('features')
axs[3].set_ylabel('samples')
axs[3].set_title('importance')
axs[3].set_yticks(range(20))
axs[3].set_xticks(range(13))
axs[3].set_xticklabels(feature_names, rotation=90)
plt.show()
axs[3].set_title('importance')

In [8]:
getattr(tabnet._model, 'tabnet_encoder')

TabNetEncoder(
  (input_bn): BatchNorm1d(13, eps=1e-05, momentum=0.03, affine=True, track_running_stats=True)
  (input_splitter): FeatureTransformer(
    (shared_block): FeatureBlock(
      (shared_layers): ModuleList(
        (0): Linear(in_features=13, out_features=16, bias=False)
      )
      (glu_blocks): ModuleList(
        (0): GLUBlock(
          (fc): Linear(in_features=13, out_features=16, bias=False)
          (gbn): GhostBatchNorm(
            (bn): BatchNorm1d(16, eps=1e-05, momentum=0.03, affine=True, track_running_stats=True)
          )
        )
      )
    )
    (indep_block): FeatureBlock(
      (glu_blocks): ModuleList(
        (0): GLUBlock(
          (fc): Linear(in_features=8, out_features=16, bias=False)
          (gbn): GhostBatchNorm(
            (bn): BatchNorm1d(16, eps=1e-05, momentum=0.03, affine=True, track_running_stats=True)
          )
        )
      )
    )
  )
  (feats_transformers): ModuleList(
    (0): FeatureTransformer(
      (shared_block): Fea